# Automating Study Area Generation for Expansion Cities

## Model Summary:

This notebook provides a workflow to generate the study area dataset described in the [Study Area definition](../docs/study-areas/README.md).

## Workflow Summary:

The notebook provides a workflow for generating study areas of expansion cities by integrating population data with urban boundary definitions. Rather than relying on administrative boundaries, the target city is selected based on its Functional Urban Area (FUA) boundary, which better reflects actual patterns of human settlement and economic activity. A 1-kilometre buffer is applied around the FUA to avoid discontinuity in setting urban boundaries. The GHS-POP reference grid, which provides a framework enabling interoperability for datasets on deprivation, is clipped to the buffered boundaries of the selected FUA. The result is a harmonised geospatial layer for each expansion city, which can be further analysed for demographic and urban expansion studies.

* **Preprocessing**: Select the target city based on FUA boundaries, which better reflect actual patterns of human settlement and economic activity than administrative boundaries, and load the GHS-POP reference grid.

* **Study Area Definition**: Apply the FUA boundary with a 1-kilometre buffer to ensure spatial continuity.

* **Spatial Clipping**: Clip the population reference grid to the buffered FUA boundary.

* **Outputs**: Export the reference grid of the selected city as GeoPackage and CSV.


### Datasets:
* [Functional Urban Area (FUA)](https://human-settlement.emergency.copernicus.eu/) - the actual urban sprawl and human activities.
* [GHS-POP grid](https://data.jrc.ec.europa.eu/dataset/2ff68a52-5b5b-4a22-8f40-c41da8332cfe) - Global Human Settlement Grid Multitemporal (1975-2030) grid.

In [1]:
# Import Libraries
import geopandas as gpd
import rasterio
from rasterio.mask import mask
from rasterio.features import shapes
from shapely.geometry import mapping, shape, box
import os
import numpy as np
import requests, zipfile, io

In [2]:
data_inputs = '../scripts/data-inputs/'

### Functional Urban Area (FUA)
The FUA was obtained from the Global Human Settlement Layer (GHSL) dataset, which provides spatial data for functional urban areas worldwide. 

In [3]:
# Define the URL for the GHS FUA data
url = "https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL//GHS_FUA_UCDB2015_GLOBE_R2019A/V1-0/GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.zip"

# Define the path to the GHS FUA data
path = f"zip+{url}!GHS_FUA_UCDB2015_GLOBE_R2019A_54009_1K_V1_0.gpkg"

GHS_FUA = gpd.read_file(path)

In [12]:
# Select Expansion City
CITY = "Accra" # Replace CITY with the name of the city
COUNTRY = "Ghana" # Replace COUNTRY with the name of the country

In [13]:
# Filter FUA for target city
fua_city = GHS_FUA[GHS_FUA["eFUA_name"] == CITY]
fua_city

,eFUA_ID,UC_num,UC_IDs,eFUA_name,Commuting,Cntry_ISO,Cntry_name,FUA_area,UC_area,FUA_p_2015,UC_p_2015,Com_p_2015,geometry
3312,2940.0,2.0,1910;1926,Accra,1.0,GHA,Ghana,1934.0,863.0,4.786265e+06,4.472469e+06,313796.012403,"MULTIPOLYGON (((-14000 724000, -11000 724000, ..."


In [14]:
# Create a buffered of the FUA
fua_city_buffered = fua_city.copy()
fua_city_buffered["geometry"] = fua_city_buffered.geometry.buffer(1000, join_style=2) # Buffer by 1000 meters
fua_union = fua_city_buffered.geometry.union_all()

### GHS-POP Reference Grid

**Note:** Directly accessing the GHS-POP reference grid from the remote [URL](https://jeodpp.jrc.ec.europa.eu/ftp/jrc-opendata/GHSL/GHS_POP_GLOBE_R2023A/GHS_POP_E2025_GLOBE_R2023A_54009_100/) may be time-consuming due to its large size.  
We recommend downloading the ZIP file locally and subsequently loading the reference grid TIFF file from disk.

In [15]:
GHS_POP = data_inputs + "GHS_POP_E2025_GLOBE_R2023A_54009_100_V1_0.tif"

In [16]:
with rasterio.open(GHS_POP) as pop_raster:

    # Ensure the FUA geometry is in the same CRS as the population raster
    fua_union = gpd.GeoSeries([fua_union], crs=fua_city.crs).to_crs(pop_raster.crs).iloc[0] 
    
    # Mask the population raster with the buffered FUA geometry
    out_image, out_transform = mask(pop_raster, [mapping(fua_union)], crop=True)

    nodata = pop_raster.nodata # keep the nodata value from the original raster
    height, width = out_image.shape[1], out_image.shape[2]
    crs = pop_raster.crs

In [17]:
pixels = []

# Iterate over each column in the raster for the current row
for row in range(height):
    for col in range(width):
        val = out_image[0, row, col] # Extract the pixel value
        if val == nodata:
            continue
        
        # Compute the spatial coordinates of the pixel corners
        minx, maxy = rasterio.transform.xy(out_transform, row, col, offset='ul')
        maxx, miny = rasterio.transform.xy(out_transform, row, col, offset='lr')
        pixel_geom = box(minx, miny, maxx, maxy)

        # Only keep pixels that intersect with the FUA polygon
        if not pixel_geom.intersects(fua_union):
            continue
        
        pixels.append({
            'geometry': pixel_geom, 
            'longitude': pixel_geom.centroid.x,  # longitude of pixel centroid
            'latitude': pixel_geom.centroid.y,
            'lon_min': minx,
            'lon_max': maxx,
            'lat_min': miny,
            'lat_max': maxy
        })

# Convert the list of pixels to a GeoDataFrame
study_area_gdf = gpd.GeoDataFrame(pixels, crs=crs)
study_area_gdf


,geometry,longitude,latitude,lon_min,lon_max,lat_min,lat_max
0,"POLYGON ((-37900 725900, -37900 726000, -38000...",-37950.0,725950.0,-38000.0,-37900.0,725900.0,726000.0
1,"POLYGON ((-37800 725900, -37800 726000, -37900...",-37850.0,725950.0,-37900.0,-37800.0,725900.0,726000.0
2,"POLYGON ((-37700 725900, -37700 726000, -37800...",-37750.0,725950.0,-37800.0,-37700.0,725900.0,726000.0
3,"POLYGON ((-37600 725900, -37600 726000, -37700...",-37650.0,725950.0,-37700.0,-37600.0,725900.0,726000.0
4,"POLYGON ((-37500 725900, -37500 726000, -37600...",-37550.0,725950.0,-37600.0,-37500.0,725900.0,726000.0
...,...,...,...,...,...,...,...
220278,"POLYGON ((-41400 672000, -41400 672100, -41500...",-41450.0,672050.0,-41500.0,-41400.0,672000.0,672100.0
220279,"POLYGON ((-41300 672000, -41300 672100, -41400...",-41350.0,672050.0,-41400.0,-41300.0,672000.0,672100.0
220280,"POLYGON ((-41200 672000, -41200 672100, -41300...",-41250.0,672050.0,-41300.0,-41200.0,672000.0,672100.0
220281,"POLYGON ((-41100 672000, -41100 672100, -41200...",-41150.0,672050.0,-41200.0,-41100.0,672000.0,672100.0


In [18]:
# Save the study area as a GeoPackage with originally CRS ESRI:54009 (World Mollweide)
study_area_gdf.to_file(f'../expansion_city/grid-boundary-{CITY.lower()}-54009.gpkg', driver="GPKG")

In [19]:
# Convert CRS to EPSG:4326
study_area = study_area_gdf.to_crs(epsg=4326)

# Save the study area as a GeoPackage
study_area.to_file(f'../expansion_city/grid-boundary-{CITY.lower()}-4326.gpkg', driver="GPKG")

In [ ]:
# Save the study area as a CSV file without geometry
study_area_gdf.drop(columns='geometry').to_csv(f'../grid-boundary-{CITY.lower()}.gpkg', index=False)